In [2]:
import os, sys
import pymysql
import pandas as pd
import subprocess
import pathlib
import mysql.connector
from datetime  import datetime 
from pprint import pprint

In [9]:
# Parameters

db = 'issues'
DataPath = './Data'

host = 'localhost'
user = 'jira'
password = 'jira'
port = '3306'

files = pathlib.Path(DataPath)# + '/*.csv', recursive=False)
list(files.glob('*.*'))

[PosixPath('Data/issuelink.sql'),
 PosixPath('Data/AO_60DB71_SPRINT.csv'),
 PosixPath('Data/AO_60DB71_SPRINT.sql'),
 PosixPath('Data/audit_changed_value.csv'),
 PosixPath('Data/audit_changed_value.sql'),
 PosixPath('Data/audit_item.csv'),
 PosixPath('Data/audit_item.sql'),
 PosixPath('Data/audit_log.csv'),
 PosixPath('Data/audit_log.sql'),
 PosixPath('Data/changegroup_changeitem.csv'),
 PosixPath('Data/changegroup_changeitem.sql'),
 PosixPath('Data/customfield.csv'),
 PosixPath('Data/customfield.sql'),
 PosixPath('Data/customfieldoption.csv'),
 PosixPath('Data/customfieldoption.sql'),
 PosixPath('Data/customfieldvalue.csv'),
 PosixPath('Data/customfieldvalue.sql'),
 PosixPath('Data/issuelink.csv'),
 PosixPath('Data/issuelinktype.csv'),
 PosixPath('Data/issuelinktype.sql'),
 PosixPath('Data/issues.db'),
 PosixPath('Data/issuestatus.csv'),
 PosixPath('Data/issuestatus.sql'),
 PosixPath('Data/issuetype.csv'),
 PosixPath('Data/issuetype.sql'),
 PosixPath('Data/jiraaction.csv'),
 PosixPath(

## Selecting

In [4]:
con = mysql.connector.connect(
      host=host,
      user=user,
      passwd=password,
      database=db
    )

In [43]:
def GetIssueAtTime(issueid, AtTime):
    changes = pd.read_sql( f"""
    select * 
    from changegroup_changeitem 
    where issueid ={issueid} and CREATED < {AtTime}
    order by groupid desc; """,con)

    issueclosed = pd.read_sql(f"""select  CREATED
    from changegroup_changeitem 
    where issueid = {issueid} and FIELD = 'status' and NEWSTRING = 'Closed'
    ORDER by CREATED DESC 
    limit 1""", con)

    issue = pd.read_sql(f"""
    select i.*, t.pname as issue_type 
    from jiraissue i
    join issuetype t on i.issuetype = t.ID 
    where i.ID = {issueid} """,con)
    
    now = pd.DataFrame(data=None, columns=changes.columns)
    req_fields = {'status':1, 'priority':1,'assignee':1,'reporter':1,'issuetype':1,'summary':1,'description':1}

    for index, row in changes.iterrows():

        #is empty
        if not req_fields:
            break
            
        if row.FIELD.lower() in req_fields:
            now = now.append(row)
            del req_fields[row.FIELD]

    result = {'created': issue.CREATED[0], 'closed':issueclosed.CREATED[0],'updated':now.CREATED.max()
              ,'summary':None, 'description':None, 'closed_date':issue.RESOLUTIONDATE[0], 'status':None, 
              'priority':None,'assignee':None,'reporter':None,'issuetype':issue.issue_type[0]}

    for index, row in now.iterrows():
        result[row.FIELD] = row.NEWSTRING if not None else  row.NEWVALUE
    return result


In [46]:
issueid = 163623
AtTime= 186461716436622

In [6]:
issueid=133474
AtTime=186925578209167

In [47]:
time(GetIssueAtTime(issueid, AtTime) )

CPU times: user 54.9 ms, sys: 0 ns, total: 54.9 ms
Wall time: 1.06 s


{'assignee': 'Tao Tran Van',
 'closed': 186521855642488.0,
 'closed_date': 186478903486432.0,
 'created': 186367238279507.0,
 'description': 'Steps:\n # Navigate to a page that has apps, for example "HIT Ø§Ù„Ù…ÙˆØ³Ù…"\n # click on apps tab\n\nActual result: The Apps section is not displaying all the apps published, although these apps are displayed in the Page Stream. for the pageÂ\xa0"HIT Ø§Ù„Ù…ÙˆØ³Ù…" there is actually 10 apps published and live, but when checking the Apps tab in Beta, it is only showing 4.Â\xa0\n\nThis case is applied on multiple other pages that has apps.\n\nExpected Results: what is published to FO should appear on both Main Stream and Apps Tab.',
 'issuetype': 'Bug',
 'priority': 'Highest',
 'reporter': None,
 'status': 'In Review',
 'summary': None,
 'updated': 186461716426622.0}